# training concept model

In [27]:
from tensorflow.keras.layers import Dense ,Conv2D, MaxPooling2D,Input
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import os
import json
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import statistics
from tensorflow.keras import backend as k
from tensorflow.keras.optimizers import RMSprop,SGD
from tensorflow.keras.models import Model,model_from_json
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from scipy.stats import weibull_min
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.utils import concordance_index
from scipy.stats import weibull_min,weibull_max
import pickle
import random
from scipy import stats
import os
from tensorflow.keras.applications import Xception
from sklearn.metrics import roc_curve
#import pickle5 as pickle

In [2]:
def file_name(file_dir):  
    for root, dirs, files in os.walk(file_dir): 
        return files

In [3]:
def folder_name(folder_name):  
    for root, dirs, files in os.walk(folder_name):  
        return dirs

In [4]:
# please change the file path

str_file=file_name('change the file path of concept')
back_file=file_name('change the file path of concept')
deb_file=file_name('change the file path of concept')
lym_file=file_name('change the file path of concept')
muc_file=file_name('change the file path of concept')
mus_file=file_name('change the file path of concept')
norm_file=file_name('change the file path of concept')
tum_file=file_name('change the file path of concept')
adi_file=file_name('change the file path of concept')

In [5]:
## all concepts

# all_fig=str_file+back_file+deb_file+lym_file+muc_file+mus_file+norm_file+tum_file+adi_file

# folder=[]
# for i in range(0,len(all_fig)):
#     if all_fig[i][3] == '-' :
#         folder.append(all_fig[i][:3])
#     else :
#         folder.append(all_fig[i][:4])

# cls=[]
# for i in range(0,len(folder)):
#     if folder[i]=='STR':
#         per=0
#         cls.append(per)
#     if folder[i]=='BACK':
#         per=1
#         cls.append(per)
#     if folder[i]=='DEB':
#         per=2
#         cls.append(per)
#     if folder[i]=='LYM':
#         per=3
#         cls.append(per)
#     if folder[i]=='MUC':
#         per=4
#         cls.append(per)
#     if folder[i]=='MUS':
#         per=5
#         cls.append(per)
#     if folder[i]=='NORM':
#         per=6
#         cls.append(per)
#     if folder[i]=='TUM':
#         per=7
#         cls.append(per)
#     if folder[i]=='ADI':
#         per=8
#         cls.append(per)


# data = {'fig':  all_fig,
#         'folder': folder,
#         'class': cls

#         }

# df = pd.DataFrame (data, columns = ['fig','folder','class'])


In [ ]:
#select 7 concepts

all_fig=str_file+deb_file+lym_file+muc_file+mus_file+tum_file+adi_file

folder=[]
for i in range(0,len(all_fig)):
    if all_fig[i][3] == '-' :
        folder.append(all_fig[i][:3])
    else :
        folder.append(all_fig[i][:4])

cls=[]
for i in range(0,len(folder)):
    if folder[i]=='STR':
        per=0
        cls.append(per)
    if folder[i]=='DEB':
        per=1
        cls.append(per)
    if folder[i]=='LYM':
        per=2
        cls.append(per)
    if folder[i]=='MUC':
        per=3
        cls.append(per)
    if folder[i]=='MUS':
        per=4
        cls.append(per)

    if folder[i]=='TUM':
        per=5
        cls.append(per)
    if folder[i]=='ADI':
        per=6
        cls.append(per)


data = {'fig':  all_fig,
        'folder': folder,
        'class': cls

        }

df = pd.DataFrame (data, columns = ['fig','folder','class'])

In [68]:
df

,fig,folder,class
0,MUC-REYWEIVF.tif,MUC,3
1,LYM-NESVKVAG.tif,LYM,2
2,DEB-LNANGSYG.tif,DEB,1
3,MUS-IHARTENR.tif,MUS,4
4,MUS-RVYEEEYM.tif,MUS,4
...,...,...,...
80666,LYM-LHCQFKPE.tif,LYM,2
80667,MUC-WGNMTSEY.tif,MUC,3
80668,MUS-MFWYMKSS.tif,MUS,4
80669,MUS-RHFCPQAV.tif,MUS,4


In [18]:

class DataGenerator(tf.keras.utils.Sequence):
    
    'Generates data for Keras'
    def __init__(self, fig_IDs,fold_name,label,folder='train', batch_size=32, dim=(299,299), n_channels=3,
                 shuffle=False):
        
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.fig_IDs = fig_IDs
        self.fold_name = fold_name
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.folder = folder
        self.label=label
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.fig_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        # Find list of IDs
        fig_IDs_temp = [self.fig_IDs[k] for k in indexes]
       
        # Generate data
        X_img,y = self.__data_generation(indexes)

        return X_img,y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.fig_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, fig_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        
        

        person_id=[]
        
        X_img = []
        y=[]
        y_per=[]

        # Generate data
        for i, ID in enumerate(fig_IDs_temp):
            # Store sample
           
            train_image=[]
            img = image.load_img('change the file path' ,target_size=(299,299,3))
#             img = image.load_img('../NCT-CRC-HE-100K/'+self.fold_name[ID]+'/' + self.fig_IDs[ID] ,target_size=(299,299,3))

            
            img = image.img_to_array(img)
            img = img/255
            train_image.append(img)
            X_img.append(train_image)
            
            
            y_per.append(self.label[ID])
            
            
        X_img = np.array(X_img).squeeze()
        
        
        
        y = np.array(y_per).astype('int')
        y = tf.convert_to_tensor(y, tf.float32)
        
 
        
        return X_img,y

In [21]:
df_valid=df[60000:]
df_valid=df_valid.reset_index(drop=True)

train_fig=df['fig'][:60000]
train_folder=df['folder'][:60000]
train_class=df['class'][:60000]

valid_fig=df_valid['fig']
valid_folder=df_valid['folder']
valid_class=df_valid['class']

In [22]:
# Generators
training_generator = DataGenerator(train_fig,train_folder,train_class) 
validation_generator = DataGenerator(valid_fig,valid_folder,valid_class)

In [23]:
base_model = Xception(weights=None,include_top=False) #weights='imagenet', include_top=False
x_out = base_model.output
gap = GlobalAveragePooling2D()(x_out)
x = Dense(1024, activation='relu')(gap)
x = Dense(len(set(df['class'])), activation='softmax')(x) # 7.9
model = Model(inputs=base_model.input, outputs=x)


In [25]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
             )

In [31]:
model.fit(training_generator,
          validation_data=validation_generator,
          epochs=1,workers=64,
         )

2024-03-14 06:52:53.366005: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1875/1875 [==============================] - ETA: 0s - loss: 0.1586 - sparse_categorical_accuracy: 0.9520

2024-03-14 06:55:51.331515: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1875/1875 [==============================] - 199s 106ms/step - loss: 0.1586 - sparse_categorical_accuracy: 0.9520 - val_loss: 0.8302 - val_sparse_categorical_accuracy: 0.8072


In [46]:
valid_prob = model.predict(validation_generator) 
#y_classes = y_prob.argmax(axis=-1)

  5/645 [..............................] - ETA: 22s

2024-03-14 07:27:45.594657: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


645/645 [==============================] - 22s 33ms/step


In [95]:
valid_data = {'fig':  [],
              'folder' :[],
#               'fb_ratio' :[],
              'STR': [],
              'DEB':[],
              'LYM':[],
              'MUC':[],
              'MUS':[],
              'TUM':[],
              'ADI':[]
            }

valid_res = pd.DataFrame (valid_data, columns = ['fig','folder',
                                                 #'fb_ratio',
                                                 'STR','DEB','LYM','MUC','MUS','TUM','ADI'])

In [96]:
per=[]
for i in range(0,len(valid_prob)):
    per+=[valid_fig[i],valid_folder[i]]
    per+=list(valid_prob[i]*100)
    valid_res.loc[len(valid_res)]=per
    per=[]
    

In [97]:
valid_res

,fig,folder,STR,DEB,LYM,MUC,MUS,TUM,ADI
0,MUC-DLLVPPRD.tif,MUC,1.655238e-03,8.961295e-04,3.507152e-06,9.995786e+01,4.797640e-03,3.390334e-02,9.005192e-04
1,STR-IEVNHHKY.tif,STR,6.335719e+00,5.600479e-02,7.445016e-07,1.032328e-02,9.358791e+01,9.988258e-03,4.080338e-05
2,ADI-KEYWHRMT.tif,ADI,3.449038e-11,1.040451e-12,9.559348e-19,8.932747e-13,2.225672e-12,2.041850e-19,1.000000e+02
3,ADI-LEWRPYII.tif,ADI,5.839502e-11,3.029886e-12,5.438264e-18,2.199994e-12,1.153645e-11,1.493662e-18,1.000000e+02
4,ADI-SAVQSAWP.tif,ADI,1.608382e-10,2.417582e-11,3.279678e-18,7.687829e-13,4.583129e-11,1.217475e-18,1.000000e+02
...,...,...,...,...,...,...,...,...,...
20635,TUM-KVYIANND.tif,TUM,1.021669e-01,8.666578e+00,1.452235e-02,3.603616e-02,3.753161e-02,9.114286e+01,3.022342e-04
20636,MUS-NCQDMFNI.tif,MUS,7.902023e-07,2.565528e-08,3.092913e-26,4.186530e-15,1.000000e+02,9.267564e-13,4.898067e-20
20637,ADI-ICMDDWNM.tif,ADI,9.902597e-02,4.215878e-01,3.911113e-03,2.385506e-01,1.756564e+01,3.438181e-02,8.163690e+01
20638,MUC-PTADKKLL.tif,MUC,7.979547e-01,8.017413e+00,4.844561e-03,8.849665e+01,3.088552e-01,9.650860e-01,1.409200e+00
